In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import warnings
import io
import math
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'numpy'

In [2]:
def moving_average(a, n=10): 
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    ret/=n
    # Masking
    ret[:n-1]=ret[n-1]
    ret=ret.reshape(-1,1)
    return ret

def read_train_patients(input_len = 8, output_len = 6):
    x_train=[]
    y_train=[]
    #y_train_f=[]
    for i in range(1, 15):
        #y_noised=[]
        
        df=pd.read_csv("Virtual_patients_20\V_CGMS_{}.csv".format(i),header=None)
        dataset=df.values
        dataset = moving_average(dataset)

        scaler = MinMaxScaler(feature_range=(0, 1)) 
        scaled_data = scaler.fit_transform(dataset)
        train_data = scaled_data
        #train_data = dataset
        
        for i in range(input_len,len(train_data)-output_len):
            x_train.append(train_data[i-input_len:i,0])
            y_train.append(train_data[i+output_len-1,0])
        
    x_train, y_train = np.array(x_train), np.array(y_train)
    #reshape 
    #x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    
    return scaler, x_train, y_train
    #return x_train, y_train

In [3]:
def train_model(x_train, y_train, batch_size=1, epochs=1):
    
    X_val = x_train[11000:]
    Y_val = y_train[11000:]
    X_train = x_train[:11000]
    Y_train = y_train[:11000]

    
    # MLP model
    model = Sequential()
    model.add(Dense(32,input_dim=8,activation="relu"))
    #model.add(Dropout(0.3))
    model.add(Dense(32,activation="relu"))
    #model.add(Dropout(0.3))
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    print(model.summary())

    model.compile(optimizer='adam', loss='mse')

    #over fitting 방지
    early_stopping = EarlyStopping(monitor='loss', patience=50, mode='auto')
    
    #fit model
    #hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping],validation_data=(X_val, Y_val))
    hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val))
    

    # 학습과정 살펴보기
    """plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.ylim(0.0, 0.15)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()"""
    return model, hist

In [4]:
scaler, x_train, y_train = read_train_patients()

In [ ]:
model, hist = train_model(x_train, y_train, batch_size=32, epochs=500) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                288       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
344/344 [==============================] - 2s 3ms/step - loss: 0.0239 - val_loss: 0.0092
Epoch 2/500
34

In [ ]:
def plot_loss(hist):
    plt.figure(figsize=(24,16))
    plt.title('Loss after smoothing',fontsize=50)
    plt.plot(hist.history['loss'], linewidth=5)
    plt.plot(hist.history['val_loss'], linewidth=5)
    plt.ylim(0, 0.0600)
    ax=plt.axes()
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.005))
    plt.xticks(fontsize=33)
    plt.yticks(fontsize=33)
    plt.ylabel('loss',fontsize=44)
    plt.xlabel('epoch',fontsize=44)
    plt.legend(['train', 'val'], loc='upper right', fontsize=44)
    plt.show()

plot_loss(hist)

In [ ]:
#이건 바꾸지 말기 - 비교해야 되니깐
def show_plots(i, continuous_ytest, continuous_predictions):
    plt.figure(figsize=(24,16))
    plt.title('Blood Glucose Prediction Model Result_patient_{}'.format(i),fontsize=40)
    plt.plot(continuous_ytest, color = 'b', linewidth=3)
    plt.plot(continuous_predictions, color = 'r', ls="--", linewidth=3)
    plt.xticks(fontsize=28)
    plt.yticks(fontsize=28)
    plt.ylim(40, 260)
    ax=plt.axes()
    ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
    plt.xlabel('Timestamp',fontsize=28)
    plt.ylabel('BG(CGM) (mg/dL)',fontsize=28)
    plt.legend(['Real','Predictions'], loc='upper right',fontsize=32)
    plt.show()

In [ ]:
def test_model(model, i, input_len=8, output_len=6):
    df = pd.read_csv("Virtual_patients_20\V_CGMS_{}.csv".format(i),header=None)
    dataset=df.values
    
    dataset = moving_average(dataset)
    
    # Scalling data from 0 - 1 to input in the neural network
    scaler = MinMaxScaler(feature_range=(0, 1)) 
    scaled_data = scaler.fit_transform(dataset)
    
    x_test =[]
    y_test =[]
    
    continuous_ytest=[]
    i= input_len
    
    for i in range(input_len,len(dataset)-output_len):
        x_test.append(scaled_data[i-input_len:i,0])
        y_test.append(dataset[i+output_len-1,0])
        continuous_ytest.append(dataset[i+output_len-1,0])

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    #y_test = np.reshape(y_test,(-1,1))
    #print("x_test: ", x_test)
    #x_test= np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
    
    predictions = model.predict(x_test)
    predictions = np.reshape(predictions,(1,-1))
    predictions = scaler.inverse_transform(predictions)
    print(predictions)
    print(y_test)
    #print("predictions: ", predictions)
    
    continuous_predictions = predictions[0]
    for i in range(1, len(predictions)):
        continuous_predictions = np.concatenate([continuous_predictions, predictions[i]])
    
    #print(predictions)
    #print(y_test)
    
    
    rmse=np.sqrt(np.mean(((predictions[0]-y_test)**2)))
    mape=np.mean(np.abs((predictions[0]-y_test)/y_test)*100)
    #print("prediction:", np.shape(pred))
    #print("y_test:",np.shape(y_test))
    #print(len(continuous_ytest))
    #print("continuouse_ytest: ", continuous_ytest)
    #print("continuous_predictions: ", continuous_predictions)
    return rmse, mape, continuous_ytest, continuous_predictions

In [ ]:
def get_correlation(arr1, arr2):
    cov = np.cov(arr1, arr2)[0, 1]  # 공분산
    arr1sd = np.std(arr1, ddof=1)   # x의 표본표준편차
    arr2sd = np.std(arr2, ddof=1)   # y의 표본표준편차
    return cov / ( arr1sd * arr2sd )
    
def get_time_gain(continuous_ytest, continuous_predictions):
    corr = np.corrcoef(continuous_ytest[6:-6], continuous_predictions[6:-6])[0,1]
    max_corr = corr
    #j=1
    max_trans = 0
    for j in range(1, 6):
        trans_corr = np.corrcoef(continuous_ytest[6:-6], continuous_predictions[6+j:-6+j])[0,1] 
        if max_corr < trans_corr:
            max_corr = trans_corr
            print(max_corr)
            max_trans = j

    return max_trans

for i in range(15,21):
    rmse, mape, continuous_ytest, continuous_predictions = test_model(model, i)
    show_plots(i, continuous_ytest, continuous_predictions)
    time_gain = get_time_gain(continuous_ytest, continuous_predictions)
    print("Time_delay(min): ", time_gain*5)
    print("Root-Mean-Squared Deviation {}".format(rmse))
    print("Mean-Absolute-Percentage-Error {}".format(mape))

In [ ]:
model.save('MLP_diabetes_model_1dropout_no_smoothing.h5')